## Imports

In [1]:
import datetime
import gc

In [2]:
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import *

In [3]:
cuda_use_gpus(0)

In [4]:
from keras import backend as K
from keras.models import Model, Sequential
from keras.layers import *
from keras.callbacks import EarlyStopping, ModelCheckpoint

Using TensorFlow backend.


## Config

In [5]:
feature_list_id = 'oofp_siamese_lstm_attention_activations'

In [6]:
NUM_FOLDS = 5
BATCH_SIZE = 512
NUM_EPOCHS = 200

In [7]:
RANDOM_SEED = 42

In [8]:
np.random.seed(RANDOM_SEED)

## Read Data

In [9]:
embedding_matrix = load(aux_data_folder + 'embedding_weights_fasttext_filtered_no_stopwords.pickle')

In [10]:
X_train_q1 = load(features_data_folder + 'X_train_nn_fasttext_q1_filtered_no_stopwords.pickle')
X_train_q2 = load(features_data_folder + 'X_train_nn_fasttext_q2_filtered_no_stopwords.pickle')

In [11]:
X_test_q1 = load(features_data_folder + 'X_test_nn_fasttext_q1_filtered_no_stopwords.pickle')
X_test_q2 = load(features_data_folder + 'X_test_nn_fasttext_q2_filtered_no_stopwords.pickle')

In [12]:
y_train = load(features_data_folder + 'y_train.pickle')

In [13]:
EMBEDDING_DIM = embedding_matrix.shape[-1]
VOCAB_LENGTH = embedding_matrix.shape[0]
MAX_SEQUENCE_LENGTH = X_train_q1.shape[-1]

In [14]:
print(EMBEDDING_DIM, VOCAB_LENGTH, MAX_SEQUENCE_LENGTH)

300 101442 30


## Train Models & Compute Out-of-Fold Predictions

In [15]:
model_params = {
    'dense_dropout_rate': 0.164,
    'lstm_dropout_rate': 0.324,
    'num_dense_1': 128,
    'num_dense_2': 64,
    'num_lstm': 256,
}

In [16]:
class AttentionWithContext(Layer):
    """
        Attention operation, with a context/query vector, for temporal data.
        Supports Masking.
        Follows the work of Yang et al. [https://www.cs.cmu.edu/~diyiy/docs/naacl16.pdf]
        "Hierarchical Attention Networks for Document Classification"
        by using a context vector to assist the attention
        # Input shape
            3D tensor with shape: `(samples, steps, features)`.
        # Output shape
            2D tensor with shape: `(samples, features)`.
        :param kwargs:
        Just put it on top of an RNN Layer (GRU/LSTM/SimpleRNN) with return_sequences=True.
        The dimensions are inferred based on the output shape of the RNN.
        Example:
            model.add(LSTM(64, return_sequences=True))
            model.add(AttentionWithContext())
        """

    def __init__(self, init='glorot_uniform', kernel_regularizer=None, bias_regularizer=None, kernel_constraint=None, bias_constraint=None,  **kwargs):
        self.supports_masking = True
        self.init = initializers.get(init)
        self.kernel_initializer = initializers.get('glorot_uniform')

        self.kernel_regularizer = regularizers.get(kernel_regularizer)
        self.bias_regularizer = regularizers.get(bias_regularizer)

        self.kernel_constraint = constraints.get(kernel_constraint)
        self.bias_constraint = constraints.get(bias_constraint)

        super(AttentionWithContext, self).__init__(**kwargs)

    def build(self, input_shape):
        self.kernel = self.add_weight((input_shape[-1], 1),
                                 initializer=self.kernel_initializer,
                                 name='{}_W'.format(self.name),
                                 regularizer=self.kernel_regularizer,
                                 constraint=self.kernel_constraint)
        self.b = self.add_weight((input_shape[1],),
                                 initializer='zero',
                                 name='{}_b'.format(self.name),
                                 regularizer=self.bias_regularizer,
                                 constraint=self.bias_constraint)

        self.u = self.add_weight((input_shape[1],),
                                 initializer=self.kernel_initializer,
                                 name='{}_u'.format(self.name),
                                 regularizer=self.kernel_regularizer,
                                 constraint=self.kernel_constraint)
        self.built = True

    def compute_mask(self, input, mask):
        return None

    def call(self, x, mask=None):
        # (x, 40, 300) x (300, 1)
        multData =  K.dot(x, self.kernel) # (x, 40, 1)
        multData = K.squeeze(multData, -1) # (x, 40)
        multData = multData + self.b # (x, 40) + (40,)

        multData = K.tanh(multData) # (x, 40)

        multData = multData * self.u # (x, 40) * (40, 1) => (x, 1)
        multData = K.exp(multData) # (X, 1)

        # apply mask after the exp. will be re-normalized next
        if mask is not None:
            mask = K.cast(mask, K.floatx()) #(x, 40)
            multData = mask*multData #(x, 40) * (x, 40, )

        # in some cases especially in the early stages of training the sum may be almost zero
        # and this results in NaN's. A workaround is to add a very small positive number ε to the sum.
        # a /= K.cast(K.sum(a, axis=1, keepdims=True), K.floatx())
        multData /= K.cast(K.sum(multData, axis=1, keepdims=True) + K.epsilon(), K.floatx())
        multData = K.expand_dims(multData)
        weighted_input = x * multData
        return K.sum(weighted_input, axis=1)


    def compute_output_shape(self, input_shape):
        return (input_shape[0], input_shape[-1],)

In [17]:
def create_dense_block(input_layer, num_units, dropout_rate):
    dense = Dense(num_units)(input_layer)
    bn = BatchNormalization()(dense)
    relu = Activation('relu')(bn)
    dropout = Dropout(dropout_rate)(relu)
    output = dropout
    
    return output

In [18]:
def zero_loss(y_true, y_pred):
    return K.zeros((1,))

In [19]:
def create_model(params):
    embedding_layer = Embedding(
        VOCAB_LENGTH,
        EMBEDDING_DIM,
        weights=[embedding_matrix],
        input_length=MAX_SEQUENCE_LENGTH,
        trainable=False,
    )
    lstm_layer = LSTM(
        params['num_lstm'],
        dropout=params['lstm_dropout_rate'],
        recurrent_dropout=params['lstm_dropout_rate'],
        return_sequences=True,
    )
    attention_layer = AttentionWithContext()

    sequence_1_input = Input(shape=(MAX_SEQUENCE_LENGTH,), dtype='int32')
    embedded_sequences_1 = embedding_layer(sequence_1_input)
    x1 = attention_layer(lstm_layer(embedded_sequences_1))

    sequence_2_input = Input(shape=(MAX_SEQUENCE_LENGTH,), dtype='int32')
    embedded_sequences_2 = embedding_layer(sequence_2_input)
    y1 = attention_layer(lstm_layer(embedded_sequences_2))

    merged = Concatenate(name='feature_output')([x1, y1])
    dropout = Dropout(params['dense_dropout_rate'])(merged)
    
    dense_1 = create_dense_block(dropout, params['num_dense_1'], params['dense_dropout_rate'])
    dense_2 = create_dense_block(dropout, params['num_dense_2'], params['dense_dropout_rate'])
    
    output = Dense(1, activation='sigmoid', name='target_output')(dense_2)

    model = Model(
        inputs=[sequence_1_input, sequence_2_input],
        outputs=[output, merged],
    )

    model.compile(
        loss={'target_output': 'binary_crossentropy', 'feature_output': zero_loss},
        loss_weights={'target_output': 1.0, 'feature_output': 0.0},
        optimizer='nadam',
        metrics=None
    )

    return model

In [20]:
feature_output_size = model_params['num_lstm'] * 2

In [21]:
model_checkpoint_path = aux_data_folder + 'fold-checkpoint-' + feature_list_id + '.h5'

In [22]:
kfold = StratifiedKFold(
    n_splits=NUM_FOLDS,
    shuffle=True,
    random_state=RANDOM_SEED
)

In [23]:
y_train_oofp = np.zeros_like(y_train, dtype='float32')
y_train_oofp_features = np.zeros((len(y_train), feature_output_size), dtype='float32')

In [ ]:
y_test_oofp = np.zeros((len(X_test_q1), NUM_FOLDS), dtype='float32')
y_test_oofp_features = np.zeros((len(X_test_q1), feature_output_size), dtype='float32')

In [ ]:
for fold_num, (ix_train, ix_val) in enumerate(kfold.split(X_train_q1, y_train)):
    X_fold_train_q1 = X_train_q1[ix_train]
    X_fold_train_q2 = X_train_q2[ix_train]

    X_fold_val_q1 = X_train_q1[ix_val]
    X_fold_val_q2 = X_train_q2[ix_val]

    y_fold_train = y_train[ix_train]
    y_fold_val = y_train[ix_val]
    
    print()
    print(f'Fitting fold {fold_num + 1} of {kfold.n_splits}')
    print()
    
    model = create_model(model_params)
    model.fit(
        [X_fold_train_q1, X_fold_train_q2],
        [y_fold_train, np.zeros((len(y_fold_train), feature_output_size))],
        
        validation_data=(
            [X_fold_val_q1, X_fold_val_q2],
            [y_fold_val, np.zeros((len(y_fold_val), feature_output_size))],
        ),

        batch_size=BATCH_SIZE,
        epochs=NUM_EPOCHS,
        verbose=1,
        
        callbacks=[
            EarlyStopping(
                monitor='val_loss',
                min_delta=0.001,
                patience=3,
                verbose=1,
                mode='auto',
            ),
            ModelCheckpoint(
                model_checkpoint_path,
                monitor='val_loss',
                save_best_only=True,
                verbose=2,
            ),
        ],
    )
        
    # Create out-of-fold prediction.
    model.load_weights(model_checkpoint_path)
    
    y_train_oofp[ix_val], y_train_oofp_features[ix_val] = model.predict(
        [X_train_q1[ix_val], X_train_q2[ix_val]],
        batch_size=1024,
        verbose=1
    )
    
    if fold_num + 1 == NUM_FOLDS:
        y_test_oofp, y_test_oofp_features = model.predict(
            [X_test_q1, X_test_q2],
            batch_size=1024,
            verbose=1
        )
    
    # Clear GPU memory.
    K.clear_session()
    del X_fold_train_q1
    del X_fold_train_q2
    del X_fold_val_q1
    del X_fold_val_q2
    del model
    gc.collect()


Fitting fold 1 of 5

Train on 323431 samples, validate on 80859 samples
Epoch 1/200
323431/323431 [==============================] - 55s - loss: 0.5458 - target_output_loss: 0.5458 - feature_output_loss: 0.0000e+00 - val_loss: 0.4893 - val_target_output_loss: 0.4893 - val_feature_output_loss: 0.0000e+00
Epoch 2/200
323431/323431 [==============================] - 63s - loss: 0.4825 - target_output_loss: 0.4825 - feature_output_loss: 0.0000e+00 - val_loss: 0.4519 - val_target_output_loss: 0.4519 - val_feature_output_loss: 0.0000e+00
Epoch 3/200
323431/323431 [==============================] - 60s - loss: 0.4507 - target_output_loss: 0.4507 - feature_output_loss: 0.0000e+00 - val_loss: 0.4171 - val_target_output_loss: 0.4171 - val_feature_output_loss: 0.0000e+00
Epoch 4/200
323431/323431 [==============================] - 61s - loss: 0.4300 - target_output_loss: 0.4300 - feature_output_loss: 0.0000e+00 - val_loss: 0.4054 - val_target_output_loss: 0.4054 - val_feature_output_loss: 0.0000

323431/323431 [==============================] - 57s - loss: 0.3389 - target_output_loss: 0.3389 - feature_output_loss: 0.0000e+00 - val_loss: 0.3595 - val_target_output_loss: 0.3595 - val_feature_output_loss: 0.0000e+00
Epoch 16/200
323431/323431 [==============================] - 57s - loss: 0.3355 - target_output_loss: 0.3355 - feature_output_loss: 0.0000e+00 - val_loss: 0.3517 - val_target_output_loss: 0.3517 - val_feature_output_loss: 0.0000e+00
Epoch 17/200
323431/323431 [==============================] - 60s - loss: 0.3324 - target_output_loss: 0.3324 - feature_output_loss: 0.0000e+00 - val_loss: 0.3553 - val_target_output_loss: 0.3553 - val_feature_output_loss: 0.0000e+00
Epoch 18/200
323431/323431 [==============================] - 57s - loss: 0.3290 - target_output_loss: 0.3290 - feature_output_loss: 0.0000e+00 - val_loss: 0.3530 - val_target_output_loss: 0.3530 - val_feature_output_loss: 0.0000e+00
Epoch 19/200
323431/323431 [==============================] - 54s - loss: 0.3

/home/yuriyguts/anaconda/lib/python3.6/site-packages/ipykernel_launcher.py:52: DeprecationWarning: assignment will raise an error in the future, most likely because your index result shape does not match the value array shape. You can use `arr.flat[index] = values` to keep the old behaviour.



Fitting fold 2 of 5

Train on 323431 samples, validate on 80859 samples
Epoch 1/200
323431/323431 [==============================] - 58s - loss: 0.5440 - target_output_loss: 0.5440 - feature_output_loss: 0.0000e+00 - val_loss: 0.4898 - val_target_output_loss: 0.4898 - val_feature_output_loss: 0.0000e+00
Epoch 2/200
323431/323431 [==============================] - 58s - loss: 0.4811 - target_output_loss: 0.4811 - feature_output_loss: 0.0000e+00 - val_loss: 0.4611 - val_target_output_loss: 0.4611 - val_feature_output_loss: 0.0000e+00
Epoch 3/200
323431/323431 [==============================] - 58s - loss: 0.4495 - target_output_loss: 0.4495 - feature_output_loss: 0.0000e+00 - val_loss: 0.4231 - val_target_output_loss: 0.4231 - val_feature_output_loss: 0.0000e+00
Epoch 4/200
323431/323431 [==============================] - 60s - loss: 0.4296 - target_output_loss: 0.4296 - feature_output_loss: 0.0000e+00 - val_loss: 0.4091 - val_target_output_loss: 0.4091 - val_feature_output_loss: 0.0000

323431/323431 [==============================] - 57s - loss: 0.3371 - target_output_loss: 0.3371 - feature_output_loss: 0.0000e+00 - val_loss: 0.3687 - val_target_output_loss: 0.3687 - val_feature_output_loss: 0.0000e+00
Epoch 17/200
323431/323431 [==============================] - 58s - loss: 0.3334 - target_output_loss: 0.3334 - feature_output_loss: 0.0000e+00 - val_loss: 0.3594 - val_target_output_loss: 0.3594 - val_feature_output_loss: 0.0000e+00
Epoch 18/200
323431/323431 [==============================] - 59s - loss: 0.3301 - target_output_loss: 0.3301 - feature_output_loss: 0.0000e+00 - val_loss: 0.3742 - val_target_output_loss: 0.3742 - val_feature_output_loss: 0.0000e+00
Epoch 19/200
323431/323431 [==============================] - 60s - loss: 0.3283 - target_output_loss: 0.3283 - feature_output_loss: 0.0000e+00 - val_loss: 0.3730 - val_target_output_loss: 0.3730 - val_feature_output_loss: 0.0000e+00
Epoch 20/200
323431/323431 [==============================] - 59s - loss: 0.3

323432/323432 [==============================] - 58s - loss: 0.3574 - target_output_loss: 0.3574 - feature_output_loss: 0.0000e+00 - val_loss: 0.3731 - val_target_output_loss: 0.3731 - val_feature_output_loss: 0.0000e+00
Epoch 12/200
323432/323432 [==============================] - 58s - loss: 0.3519 - target_output_loss: 0.3519 - feature_output_loss: 0.0000e+00 - val_loss: 0.3750 - val_target_output_loss: 0.3750 - val_feature_output_loss: 0.0000e+00
Epoch 13/200
323432/323432 [==============================] - 57s - loss: 0.3477 - target_output_loss: 0.3477 - feature_output_loss: 0.0000e+00 - val_loss: 0.3736 - val_target_output_loss: 0.3736 - val_feature_output_loss: 0.0000e+00
Epoch 14/200
323432/323432 [==============================] - 56s - loss: 0.3434 - target_output_loss: 0.3434 - feature_output_loss: 0.0000e+00 - val_loss: 0.3639 - val_target_output_loss: 0.3639 - val_feature_output_loss: 0.0000e+00
Epoch 15/200
323432/323432 [==============================] - 55s - loss: 0.3

323433/323433 [==============================] - 53s - loss: 0.4830 - target_output_loss: 0.4830 - feature_output_loss: 0.0000e+00 - val_loss: 0.4429 - val_target_output_loss: 0.4429 - val_feature_output_loss: 0.0000e+00
Epoch 3/200
323433/323433 [==============================] - 53s - loss: 0.4534 - target_output_loss: 0.4534 - feature_output_loss: 0.0000e+00 - val_loss: 0.4208 - val_target_output_loss: 0.4208 - val_feature_output_loss: 0.0000e+00
Epoch 4/200
323433/323433 [==============================] - 56s - loss: 0.4323 - target_output_loss: 0.4323 - feature_output_loss: 0.0000e+00 - val_loss: 0.4141 - val_target_output_loss: 0.4141 - val_feature_output_loss: 0.0000e+00
Epoch 5/200
323433/323433 [==============================] - 58s - loss: 0.4158 - target_output_loss: 0.4158 - feature_output_loss: 0.0000e+00 - val_loss: 0.4003 - val_target_output_loss: 0.4003 - val_feature_output_loss: 0.0000e+00
Epoch 6/200
323433/323433 [==============================] - 58s - loss: 0.4028 

323433/323433 [==============================] - 59s - loss: 0.3327 - target_output_loss: 0.3327 - feature_output_loss: 0.0000e+00 - val_loss: 0.3568 - val_target_output_loss: 0.3568 - val_feature_output_loss: 0.0000e+00
Epoch 19/200
323433/323433 [==============================] - 59s - loss: 0.3302 - target_output_loss: 0.3302 - feature_output_loss: 0.0000e+00 - val_loss: 0.3545 - val_target_output_loss: 0.3545 - val_feature_output_loss: 0.0000e+00
Epoch 20/200
323433/323433 [==============================] - 60s - loss: 0.3274 - target_output_loss: 0.3274 - feature_output_loss: 0.0000e+00 - val_loss: 0.3649 - val_target_output_loss: 0.3649 - val_feature_output_loss: 0.0000e+00
Epoch 21/200
323433/323433 [==============================] - 58s - loss: 0.3251 - target_output_loss: 0.3251 - feature_output_loss: 0.0000e+00 - val_loss: 0.3572 - val_target_output_loss: 0.3572 - val_feature_output_loss: 0.0000e+00
Epoch 22/200
323433/323433 [==============================] - 55s - loss: 0.3

In [ ]:
cv_score = log_loss(y_train, y_train_oofp)
print('CV score:', cv_score)

## Save feature names

In [ ]:
feature_names = [
    'oofp_siamese_lstm_attention_activations',
]

In [ ]:
save_feature_names(feature_names, feature_list_id)

## Save Train features

In [ ]:
save_feature_list(y_train_oofp_features, 'train', feature_list_id)

## Save Test features

In [ ]:
save_feature_list(y_test_oofp_features, 'test', feature_list_id)

## Explore

In [ ]:
pd.DataFrame(train_features).describe().T

In [ ]:
pd.DataFrame(test_features).describe().T